<a href="https://colab.research.google.com/github/Zakedu/workbook/blob/main/xray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. 실험환경 셋업

Batch size, Epoch 등을 변경

In [ ]:
import re
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

ROOT_PATH = os.path.join("/content/drive")

BATCH_SIZE = 16

IMAGE_SIZE = [180,180]

EPOCHS = 25

print(ROOT_PATH)


/content/drive


# 2. 데이터 준비
원본 데이터 : 전처리 및 배치 구성
좌우반전?

In [ ]:
train_filenames = tf.io.gfile.glob(str(ROOT_PATH + '/chest_xray/train/*/*'))
test_filenames = tf.io.gfile.glob(str(ROOT_PATH + '/chest_xray/test/*/*'))
val_filenames = tf.io.gfile.glob(str(ROOT_PATH + '/chest_xray/val/*/*'))
​
print(len(train_filenames))
print(len(test_filenames))
print(len(val_filenames))


In [ ]:
filenames = tf.io.gfile.glob(str(ROOT_PATH + '/chest_xray/train/*/*'))
filenames.extend(tf.io.gfile.glob(str(ROOT_PATH + '/chest_xray/val/*/*')))
train_filenames, val_filenames = train_test_split(filenames, test_size=0.2)
​
print(len(train_filenames))
print(len(val_filenames))


In [ ]:
COUNT_NORMAL = len([filename for filename in train_filenames if "NORMAL" in filename])
print("Normal images count in training set: " + str(COUNT_NORMAL))
​
COUNT_PNEUMONIA = len([filename for filename in train_filenames if "PNEUMONIA" in filename])
print("Pneumonia images count in training set: " + str(COUNT_PNEUMONIA))


In [ ]:
#좌우반전
​def augment(image,label):
   image = tf.image.random_flip_left_right(image)  # 랜덤하게 좌우를 반전합니다.
   return image,label
​
def prepare_for_training(ds, shuffle_buffer_size=1000):
   # augment 적용 부분이 배치처리 함수에 추가되었습니다.
   ds = ds.map(
           augment,       # augment 함수 적용
           num_parallel_calls=2
       )
​
   ds = ds.shuffle(buffer_size=shuffle_buffer_size)
​
   ds = ds.repeat()
​
   ds = ds.batch(BATCH_SIZE)
​
   ds = ds.prefetch(buffer_size=AUTOTUNE)
​
   return ds
​
train_ds = prepare_for_training(train_ds)
val_ds = prepare_for_training(val_ds)


# 3. 데이터 시각화

학습용 데이터를 시각화해서 확인해 봅니다.
만약 augmentation을 시도했다면 이후 실습코드에 있는 show_batch() 함수를 통해 실제로 좌우반전 등이 제대로 처리되었는지 확인


In [ ]:
image_batch, label_batch = next(iter(train_ds))

def show_batch(image_batch, lable_batch):
  plt.figure(figsize=(10,10))
  for i in range(16):
    ax = plt.subplot(5,5,n+1)
    plt.imshow(image_batch[n])
    if label_batch[n]:
      plt.title("PNEUMONIA")
    else:
      plt.title("NORNMAL")
    plt.axis("off")

show_batch(image_batch.numpy(), lable_batch.numpy())

NameError: ignored

# 4. CNN 모델링
의료영상 판독을 위해 실습에서 구현했던 model에서 다양한 것들을 바꾸어 가며 실험해볼 수 있습니다. Convolution filter, 채널 개수, activation, 모델구조 등을 다양하게 바꾸어볼 수 있습니다.
그리고, 우리는 BatchNormalization과 Dropout을 한 모델 안에서 동시에 사용하는 특이한 구성을 실습했습니다.
이것은 일반적으로 잘 사용되는 형태는 아닙니다. 하지만 이미지 사이즈가 크고 데이터가 부족한 의료영상에서는 실용적으로 간혹 좋은 성능을 보이기도 합니다. 만약 이 구성을 변경해 보면 어떤 효과가 발생하는지도 실험해 봅시다. BatchNormalization을 쓰거나 혹은 쓰지 않거나, Dropout을 쓰거나 혹은 쓰지 않거나 할 수 있습니다. 또, Dropout 비율을 변경해볼 수도 있습니다.


In [ ]:
def conv_block(filters):
  block = tf.keras.Sequential([
    tf.keras.layers.SeparableConv2D(filters, 3, activation='relu', padding='same'),
    tf.keras.layers.SeparableConv2D(filters, 3, activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D()
  ]
  )

  return block
  

# 5. 데이터 불균형 처리

실습코드에서 데이터의 imbalance 문제에 대처하기 위해 데이터 비율로 나누어진 class_weight를 설정해 주었습니다. 만약 이러한 처리를 생략한다면 어떻게 될까요? 또 recall을 강조하기 위해 폐렴데이터를 잘 맞추는 것을 더 강화하는 효과를 만들어낼 수는 없을까요?


# 6. 모델 훈련
loss 함수를 변경하기는 어렵겠지만, optimizer나 learning rate 등의 변화를 고려해볼 수 있을 것입니다.


# 7. 결과 확인 및 시각화
테스트데이터로 훈련된 모델을 평가해 봅시다. 우선은 accuracy를 고려해야겠지만 의료영상 모델의 특성상 recall도 중요합니다. 훈련과정의 history 그래프를 시각화해 보고, 학습 진행양상을 면밀히 분석해 보는 것도 잊지 않도록 합시다.
